# جنریٹو نیٹ ورکس

ریکرنٹ نیورل نیٹ ورکس (RNNs) اور ان کے گیٹڈ سیل ویریئنٹس جیسے کہ لانگ شارٹ ٹرم میموری سیلز (LSTMs) اور گیٹڈ ریکرنٹ یونٹس (GRUs) نے زبان ماڈلنگ کے لیے ایک طریقہ فراہم کیا، یعنی یہ الفاظ کی ترتیب سیکھ سکتے ہیں اور کسی ترتیب میں اگلے لفظ کی پیش گوئی کر سکتے ہیں۔ اس سے ہمیں RNNs کو **جنریٹو کاموں** کے لیے استعمال کرنے کی اجازت ملتی ہے، جیسے عام متن کی تخلیق، مشین ترجمہ، اور یہاں تک کہ تصویر کی وضاحت۔

پچھلے یونٹ میں زیر بحث RNN آرکیٹیکچر میں، ہر RNN یونٹ اگلی چھپی ہوئی حالت کو آؤٹ پٹ کے طور پر پیدا کرتا تھا۔ تاہم، ہم ہر ریکرنٹ یونٹ میں ایک اور آؤٹ پٹ بھی شامل کر سکتے ہیں، جو ہمیں ایک **ترتیب** آؤٹ پٹ کرنے کی اجازت دے گا (جو اصل ترتیب کے برابر لمبائی میں ہوگی)۔ مزید برآں، ہم ایسے RNN یونٹس استعمال کر سکتے ہیں جو ہر قدم پر ان پٹ قبول نہیں کرتے، بلکہ صرف ایک ابتدائی حالت ویکٹر لیتے ہیں، اور پھر آؤٹ پٹس کی ایک ترتیب پیدا کرتے ہیں۔

اس نوٹ بک میں، ہم سادہ جنریٹو ماڈلز پر توجہ مرکوز کریں گے جو ہمیں متن تخلیق کرنے میں مدد دیتے ہیں۔ سادگی کے لیے، آئیے ایک **کریکٹر لیول نیٹ ورک** بنائیں، جو حرف بہ حرف متن تخلیق کرتا ہے۔ تربیت کے دوران، ہمیں کچھ متن کا مجموعہ لینا ہوگا اور اسے حرفوں کی ترتیب میں تقسیم کرنا ہوگا۔


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## کردار کی لغت بنانا

کردار کی سطح پر جنریٹو نیٹ ورک بنانے کے لیے، ہمیں متن کو الفاظ کے بجائے انفرادی کرداروں میں تقسیم کرنا ہوگا۔ `TextVectorization` لیئر، جسے ہم پہلے استعمال کر رہے تھے، یہ کام نہیں کر سکتی، اس لیے ہمارے پاس دو اختیارات ہیں:

* متن کو دستی طور پر لوڈ کریں اور خود سے ٹوکنائزیشن کریں، جیسا کہ [Keras کی اس آفیشل مثال](https://keras.io/examples/generative/lstm_character_level_text_generation/) میں دکھایا گیا ہے۔
* کردار کی سطح پر ٹوکنائزیشن کے لیے `Tokenizer` کلاس استعمال کریں۔

ہم دوسرے آپشن کے ساتھ جائیں گے۔ `Tokenizer` کو الفاظ میں ٹوکنائز کرنے کے لیے بھی استعمال کیا جا سکتا ہے، لہذا کردار کی سطح سے الفاظ کی سطح پر ٹوکنائزیشن میں آسانی سے سوئچ کیا جا سکتا ہے۔

کردار کی سطح پر ٹوکنائزیشن کرنے کے لیے، ہمیں `char_level=True` پیرامیٹر پاس کرنا ہوگا:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

ہم ایک خاص ٹوکن استعمال کرنا چاہتے ہیں تاکہ **تسلسل کے اختتام** کو ظاہر کیا جا سکے، جسے ہم `<eos>` کہیں گے۔ آئیے اسے دستی طور پر ذخیرہ الفاظ میں شامل کریں:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

اب، متن کو نمبروں کے سلسلوں میں تبدیل کرنے کے لیے، ہم استعمال کر سکتے ہیں:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## عنوانات بنانے کے لیے ایک جنریٹو RNN کی تربیت

ہم RNN کو خبروں کے عنوانات بنانے کے لیے اس طرح تربیت دیں گے۔ ہر مرحلے پر، ہم ایک عنوان لیں گے، جسے RNN میں فیڈ کیا جائے گا، اور ہر ان پٹ کردار کے لیے ہم نیٹ ورک سے اگلا آؤٹ پٹ کردار پیدا کرنے کو کہیں گے:

![تصویر جو 'HELLO' لفظ کے RNN جنریشن کی مثال دکھا رہی ہے۔](../../../../../translated_images/ur/rnn-generate.56c54afb52f9781d.webp)

ہمارے سلسلے کے آخری کردار کے لیے، ہم نیٹ ورک سے `<eos>` ٹوکن پیدا کرنے کو کہیں گے۔

یہاں استعمال ہونے والے جنریٹو RNN اور دیگر RNN کے درمیان بنیادی فرق یہ ہے کہ ہم RNN کے ہر مرحلے سے آؤٹ پٹ لیں گے، نہ کہ صرف آخری سیل سے۔ یہ `return_sequences` پیرامیٹر کو RNN سیل میں مخصوص کر کے حاصل کیا جا سکتا ہے۔

لہٰذا، تربیت کے دوران، نیٹ ورک کو ان پٹ ایک خاص لمبائی کے انکوڈ شدہ کرداروں کا سلسلہ ہوگا، اور آؤٹ پٹ بھی اتنی ہی لمبائی کا ایک سلسلہ ہوگا، لیکن ایک عنصر سے شفٹ کیا ہوا اور `<eos>` پر ختم ہوگا۔ ایک منی بیچ میں ایسے کئی سلسلے شامل ہوں گے، اور ہمیں تمام سلسلوں کو سیدھ میں لانے کے لیے **پیڈنگ** کا استعمال کرنا ہوگا۔

آئیے ایسی فنکشنز بناتے ہیں جو ہمارے لیے ڈیٹاسیٹ کو تبدیل کریں۔ چونکہ ہم منی بیچ کی سطح پر سلسلوں کو پیڈ کرنا چاہتے ہیں، اس لیے ہم پہلے `.batch()` کال کر کے ڈیٹاسیٹ کو بیچ کریں گے، اور پھر اسے تبدیل کرنے کے لیے `map` کا استعمال کریں گے۔ اس طرح، تبدیلی کا فنکشن پورے منی بیچ کو ایک پیرامیٹر کے طور پر لے گا:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

یہاں ہم کچھ اہم کام کرتے ہیں:

* ہم پہلے اسٹرنگ ٹینسر سے اصل متن نکالتے ہیں
* `text_to_sequences` اسٹرنگز کی فہرست کو عددی ٹینسرز کی فہرست میں تبدیل کرتا ہے
* `pad_sequences` ان ٹینسرز کو ان کی زیادہ سے زیادہ لمبائی تک بڑھاتا ہے
* آخر میں ہم تمام کرداروں کو ون-ہاٹ انکوڈ کرتے ہیں، اور ساتھ ہی شفٹنگ اور `<eos>` شامل کرتے ہیں۔ جلد ہی ہم دیکھیں گے کہ ہمیں ون-ہاٹ انکوڈ کردہ کرداروں کی ضرورت کیوں ہے

تاہم، یہ فنکشن **Pythonic** ہے، یعنی اسے Tensorflow کے کمپیوٹیشنل گراف میں خودکار طور پر تبدیل نہیں کیا جا سکتا۔ اگر ہم اس فنکشن کو براہ راست `Dataset.map` فنکشن میں استعمال کرنے کی کوشش کریں تو ہمیں غلطیاں ملیں گی۔ ہمیں اس Pythonic کال کو `py_function` ریپر استعمال کرتے ہوئے بند کرنے کی ضرورت ہے:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **نوٹ**: Pythonic اور Tensorflow transformation فنکشنز کے درمیان فرق کرنا تھوڑا پیچیدہ لگ سکتا ہے، اور آپ سوچ رہے ہوں گے کہ ہم ڈیٹا سیٹ کو `fit` میں دینے سے پہلے standard Python فنکشنز کے ذریعے کیوں تبدیل نہیں کرتے۔ اگرچہ یہ ممکن ہے، لیکن `Dataset.map` استعمال کرنے کا ایک بڑا فائدہ ہے، کیونکہ ڈیٹا transformation pipeline Tensorflow computational graph کے ذریعے چلائی جاتی ہے، جو GPU computations کا فائدہ اٹھاتی ہے اور CPU/GPU کے درمیان ڈیٹا منتقل کرنے کی ضرورت کو کم کرتی ہے۔

اب ہم اپنا generator نیٹ ورک بنا سکتے ہیں اور تربیت شروع کر سکتے ہیں۔ یہ کسی بھی recurrent cell پر مبنی ہو سکتا ہے جس پر ہم نے پچھلے یونٹ میں بات کی تھی (simple, LSTM یا GRU)۔ ہماری مثال میں ہم LSTM استعمال کریں گے۔

چونکہ نیٹ ورک characters کو input کے طور پر لیتا ہے، اور vocabulary کا سائز کافی چھوٹا ہے، ہمیں embedding layer کی ضرورت نہیں ہے، one-hot-encoded input براہ راست LSTM cell میں جا سکتا ہے۔ Output layer ایک `Dense` classifier ہوگا جو LSTM output کو one-hot-encoded token numbers میں تبدیل کرے گا۔

اس کے علاوہ، چونکہ ہم variable-length sequences کے ساتھ کام کر رہے ہیں، ہم `Masking` layer استعمال کر سکتے ہیں تاکہ ایک ماسک بنایا جا سکے جو string کے padded حصے کو نظر انداز کرے۔ یہ سختی سے ضروری نہیں ہے، کیونکہ ہم `<eos>` token سے آگے کی چیزوں میں زیادہ دلچسپی نہیں رکھتے، لیکن ہم اس layer type کے ساتھ کچھ تجربہ حاصل کرنے کے لیے اسے استعمال کریں گے۔ `input_shape` `(None, vocab_size)` ہوگا، جہاں `None` variable length sequence کو ظاہر کرتا ہے، اور output shape بھی `(None, vocab_size)` ہوگا، جیسا کہ آپ `summary` سے دیکھ سکتے ہیں۔


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## آؤٹ پٹ تیار کرنا

اب جب کہ ہم نے ماڈل کو تربیت دے دی ہے، ہم اسے کچھ آؤٹ پٹ تیار کرنے کے لیے استعمال کرنا چاہتے ہیں۔ سب سے پہلے، ہمیں ایک طریقہ چاہیے جس سے ہم ٹوکن نمبروں کی ترتیب سے ظاہر کردہ متن کو ڈی کوڈ کر سکیں۔ اس کے لیے، ہم `tokenizer.sequences_to_texts` فنکشن استعمال کر سکتے ہیں؛ تاہم، یہ کردار کی سطح پر ٹوکنائزیشن کے ساتھ اچھی طرح کام نہیں کرتا۔ اس لیے ہم ٹوکنز کی ایک ڈکشنری (جسے `word_index` کہا جاتا ہے) لیں گے، ایک ریورس میپ بنائیں گے، اور اپنی ڈی کوڈنگ فنکشن لکھیں گے:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

آئیے جنریشن شروع کرتے ہیں۔ ہم پہلے ایک سٹرنگ `start` لیں گے، اسے ایک سیکوئنس `inp` میں انکوڈ کریں گے، اور پھر ہر مرحلے پر ہم اپنے نیٹ ورک کو کال کریں گے تاکہ اگلا کردار معلوم کیا جا سکے۔

نیٹ ورک کا آؤٹ پٹ `out` ایک ویکٹر ہے جس میں `vocab_size` عناصر ہوتے ہیں جو ہر ٹوکن کے امکانات کی نمائندگی کرتے ہیں، اور ہم `argmax` استعمال کرکے سب سے زیادہ ممکنہ ٹوکن نمبر تلاش کر سکتے ہیں۔ پھر ہم اس کردار کو جنریٹ کیے گئے ٹوکنز کی فہرست میں شامل کرتے ہیں اور جنریشن کے ساتھ آگے بڑھتے ہیں۔ ایک کردار جنریٹ کرنے کا یہ عمل `size` بار دہرایا جاتا ہے تاکہ مطلوبہ تعداد میں کردار جنریٹ کیے جا سکیں، اور ہم جلدی ختم کرتے ہیں جب `eos_token` سامنے آتا ہے۔


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## تربیت کے دوران آؤٹ پٹ کا نمونہ لینا

چونکہ ہمارے پاس کوئی مفید میٹرکس جیسے *درستگی* موجود نہیں ہے، ہمارے ماڈل کے بہتر ہونے کا واحد طریقہ یہ ہے کہ تربیت کے دوران **نمونہ** لے کر پیدا کردہ اسٹرنگ کو دیکھیں۔ اس کے لیے، ہم **کال بیکس** استعمال کریں گے، یعنی وہ فنکشنز جنہیں ہم `fit` فنکشن میں پاس کر سکتے ہیں، اور جو تربیت کے دوران وقتاً فوقتاً بلائے جائیں گے۔


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


یہ مثال پہلے ہی کافی اچھا متن تیار کرتی ہے، لیکن اسے کئی طریقوں سے مزید بہتر بنایا جا سکتا ہے:

* **زیادہ متن**۔ ہم نے اپنے کام کے لیے صرف عنوانات استعمال کیے ہیں، لیکن آپ مکمل متن کے ساتھ تجربہ کرنا چاہ سکتے ہیں۔ یاد رکھیں کہ RNNs لمبے سلسلوں کو سنبھالنے میں زیادہ اچھے نہیں ہوتے، اس لیے یہ بہتر ہوگا کہ یا تو انہیں چھوٹے جملوں میں تقسیم کریں، یا ہمیشہ کسی پہلے سے طے شدہ قدر `num_chars` (مثلاً، 256) کی ایک مقررہ ترتیب کی لمبائی پر تربیت کریں۔ آپ اوپر دی گئی مثال کو ایسی ساخت میں تبدیل کرنے کی کوشش کر سکتے ہیں، [Keras کا سرکاری ٹیوٹوریل](https://keras.io/examples/generative/lstm_character_level_text_generation/) کو بطور تحریک استعمال کرتے ہوئے۔

* **ملٹی لیئر LSTM**۔ 2 یا 3 LSTM سیلز کی تہوں کو آزمانا ایک اچھا خیال ہو سکتا ہے۔ جیسا کہ ہم نے پچھلے یونٹ میں ذکر کیا تھا، LSTM کی ہر تہہ متن سے مخصوص پیٹرنز نکالتی ہے، اور کردار کی سطح کے جنریٹر کے معاملے میں ہم توقع کر سکتے ہیں کہ نچلی LSTM سطح حرفوں کو نکالنے کی ذمہ دار ہوگی، اور اعلیٰ سطحیں الفاظ اور الفاظ کے مجموعے کے لیے۔ یہ آسانی سے LSTM کنسٹرکٹر کو تہوں کی تعداد کے پیرامیٹر کو پاس کر کے نافذ کیا جا سکتا ہے۔

* آپ **GRU یونٹس** کے ساتھ بھی تجربہ کرنا چاہ سکتے ہیں اور دیکھ سکتے ہیں کہ کون سے بہتر کارکردگی دکھاتے ہیں، اور **مختلف چھپی ہوئی تہہ کے سائز** کے ساتھ بھی۔ بہت بڑی چھپی ہوئی تہہ زیادہ فٹنگ کا سبب بن سکتی ہے (مثلاً، نیٹ ورک بالکل وہی متن سیکھ لے گا)، اور چھوٹے سائز سے اچھا نتیجہ پیدا نہ ہو سکے۔


## نرم متن کی تخلیق اور درجہ حرارت

`generate` کی پچھلی تعریف میں، ہم ہمیشہ اس کردار کو منتخب کر رہے تھے جس کی سب سے زیادہ امکانیت ہو، تاکہ اسے پیدا شدہ متن میں اگلا کردار بنایا جا سکے۔ اس کا نتیجہ یہ نکلا کہ متن اکثر ایک ہی کردار کے سلسلوں کے درمیان "چکر" لگاتا رہا، جیسے اس مثال میں:
```
today of the second the company and a second the company ...
```

تاہم، اگر ہم اگلے کردار کے لیے امکانیت کی تقسیم کو دیکھیں، تو یہ ممکن ہے کہ چند سب سے زیادہ امکانیتوں کے درمیان فرق زیادہ نہ ہو، مثلاً ایک کردار کی امکانیت 0.2 ہو سکتی ہے، اور دوسرے کی 0.19، وغیرہ۔ مثال کے طور پر، جب '*play*' کے سلسلے میں اگلے کردار کی تلاش کی جائے، تو اگلا کردار برابر طور پر خالی جگہ یا **e** ہو سکتا ہے (جیسے لفظ *player* میں)۔

یہ ہمیں اس نتیجے پر پہنچاتا ہے کہ ہمیشہ سب سے زیادہ امکانیت والے کردار کو منتخب کرنا "منصفانہ" نہیں ہوتا، کیونکہ دوسرے سب سے زیادہ امکانیت والے کردار کو منتخب کرنا بھی ہمیں بامعنی متن کی طرف لے جا سکتا ہے۔ زیادہ دانشمندی یہ ہے کہ نیٹ ورک کے آؤٹ پٹ سے دی گئی امکانیت کی تقسیم سے کرداروں کو **نمونہ** کے طور پر منتخب کیا جائے۔

یہ نمونہ `np.multinomial` فنکشن کے ذریعے کیا جا سکتا ہے، جو کہ **ملٹی نومیئل تقسیم** کو نافذ کرتا ہے۔ ایک فنکشن جو اس **نرم** متن کی تخلیق کو نافذ کرتا ہے، نیچے بیان کیا گیا ہے:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

ہم نے ایک اور پیرامیٹر متعارف کرایا ہے جسے **temperature** کہا جاتا ہے، جو یہ ظاہر کرنے کے لیے استعمال ہوتا ہے کہ ہمیں سب سے زیادہ احتمال پر کتنی سختی سے قائم رہنا چاہیے۔ اگر temperature 1.0 ہو، تو ہم منصفانہ ملٹی نومیئل سیمپلنگ کرتے ہیں، اور جب temperature لامحدود ہو جاتا ہے - تمام احتمالات برابر ہو جاتے ہیں، اور ہم اگلا کردار بے ترتیب منتخب کرتے ہیں۔ نیچے دیے گئے مثال میں ہم مشاہدہ کر سکتے ہیں کہ جب ہم temperature کو بہت زیادہ بڑھاتے ہیں تو متن بے معنی ہو جاتا ہے، اور جب یہ 0 کے قریب ہو جاتا ہے تو یہ "cycled" سخت پیدا کردہ متن سے مشابہت رکھتا ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
